In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import StructField,StructType,IntegerType,StringType
schema=StructType([\
    StructField('emp_id',IntegerType()),\
        StructField('name',StringType()),\
            StructField('city',StringType()),\
                StructField('country',StringType())])
data=[(1000,'Michael','columbus','lanka')]
df=spark.createDataFrame(data=data,schema=schema)
display(df)

emp_id,name,city,country
1000,Michael,columbus,lanka


In [0]:
%sql
create or replace table dim_emp(
  emp_id int,
  name string,
  city string,
  country string
)using delta
location '/FileStore/tables/delta_merge'


In [0]:
%sql
select * from dim_emp

emp_id,name,city,country
1000,Michael,columbus,lanka
2000,Michael,chiago,USA
null,Michael,chiago,USA
null,Michael,chiago,USA


In [0]:
df.createOrReplaceTempView('sourceview')

In [0]:
%sql
select * from sourceview

emp_id,name,city,country
1000,Michael,columbus,lanka


In [0]:
%sql
merge into dim_emp as target
using sourceview as source
on target.emp_id=source.emp_id
when matched then update set 
target.emp_id=source.emp_id,
target.name=source.name,
target.city=source.city,
target.country=source.country
when not matched then
insert (emp_id,name,city,country) values(emp_id,name,city,country)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,1,0,0


In [0]:
data=[(2000,'Michael','chiago','USA')]
df=spark.createDataFrame(data=data,schema=schema)
display(df)

emp_id,name,city,country
2000,Michael,chiago,USA


In [0]:
df.createOrReplaceTempView('source_vi')

In [0]:
%sql
select * from source_vi

emp_id,name,city,country
2000,Michael,chiago,USA


In [0]:
from pyspark.sql.functions import *
from delta.tables import *
delta_df=DeltaTable.forPath(spark,'/FileStore/tables/delta_merge')

In [0]:
delta_df.alias('target').merge(source=df.alias('source'),
condition='target.emp_id=source.emp_id').whenMatchedUpdate(set=
                                                           {'name':"source.name",
                                                              'city':"source.city",
                                                              'country':"source.country" 
                                                           }).whenNotMatchedInsert(values=
                                                                                   {
                                                                                       'emp_id':"source.emp_id",
                                                                                       'name':"source.name",
                                                              'city':"source.city",
                                                              'country':"source.country" 
                                                                                       
                                                                                   }).execute()